# Model training and predictions

- After splitting the data into training and test data, in the second part of the exercise, you'll train the `ALS` algorithm using the training data. PySpark MLlib's ALS algorithm has the following mandatory parameters - `rank` (the number of latent factors in the model) and `iterations` (number of iterations to run). After training the ALS model, you can use the `model` to predict the ratings from the test data. For this, you will provide the user and item columns from the test dataset and finally print the first 2 rows of `predictAll()` output.

- Remember, you have `SparkContext` `sc`, `training_data` and `test_data` are already available in your workspace.

## Instructions

- Train ALS algorithm with training data and configured parameters (rank = 10 and iterations = 10).
- Drop the rating column in the test data.
- Test the model by predicting the rating from the test data.
- Print the first two rows of the predicted ratings.

In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [4]:
from pyspark.mllib.recommendation import Rating
from pyspark.mllib.recommendation import ALS

file_path = 'file:///home/talentum/test-jupyter/P4/2_CollaborativeFiltering/Dataset/ratings.csv'

# Load the data into RDD
data = sc.textFile(file_path) # 100004 records

# Split the RDD 
ratings = data.map(lambda l: l.split(','))

# Transform the ratings RDD 
ratings_final = ratings.map(lambda line: Rating(int(line[0]), int(line[1]), float(line[2])))

# Split the data into training and test
training_data, test_data = ratings_final.randomSplit([0.8, 0.2])

# Create the ALS model on the training data
model = ALS.train( ratings_final, rank=10, iterations=10)

# Drop the ratings column 
testdata_no_rating = test_data.map(lambda p: (p[0], p[1]))

# Predict the model  
predictions = model.predictAll(testdata_no_rating)

# Print the first rows of the RDD
predictions.take(2)


[Rating(user=217, product=315, rating=1.1740388978124647),
 Rating(user=385, product=315, rating=2.447395138074653)]